# PyMySQL

纯python实现的MySQL客户端操作数据库，支持事务、存储过程、批量执行等

## 连接数据库

pymysql.connect()
返回connection对象

In [4]:
import pymysql

connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')
connection.close()

### connection对象支持的方法

cursor() 使用该连接创建并返回游标

commit() 提交事务

rollback() 回滚当前事务

close() 关闭连接

## 游标 cursor

### 什么是游标

有时候，我们执行一条查询语句的时候，往往会得到N条返回结果，执行sql语句取出这些返回结果的接口(起始点)，就是游标。沿着这个游标，我们可以一次取出一行记录

当不使用游标功能，我们去执行 select * from student where age > 20; 这条语句的时候，如果有1000条返回结果，系统会一次性将1000条记录返回到界面 中，你没有选择，也不能做其他操作

当我们开启了游标功能，再去执行这条语句的时候，系统会先帮你找到这些行， 先给你存放起来，然后提供了一个游标接口。当你需要数据的时候，就借助这个游标去一行行的取出数据，你每取出一条记录，游标指针就朝前移动一次，一直到取完最后一行数据后

如果不使用游标功能，直接使用select查询，会一次性将结果集打印到屏幕上，你无法针对结果集做第二次编程

使用游标功能后，我们可以将得到的结果先保存起来，然后可以随意进行自己的编程，得到我们最终想要的结果集

### python中的游标

在pymysql中操作数据库，就是使用游标这种方式来获取表中的数据


**使用游标的操作步骤:**

- 首先，使用pymysql连接上mysql数据库，得到一个数据库对象。然后，我们必须要开启数据库中的游标功能，得到一个游标对象。

- 接着，使用游标对象中的execute()方法，去执行某个SQL语句，系统会根据你的SQL语句，找到这些匹配行，给你存储起来，而不是一次性的打印到屏幕上。什么时候需要这个结果中的数据的时候，你就去获取它。

- 最后，就是获取结果集中的数据了，这里有两种方法获取结果集中的数据。一个是fetchone()，该方法一次获取一条记录，每一条记录是一个元组形式的数据，每获取一条记录游标会往前移动一格，等待获取下一条记录；一个是fetchall()方法，能够一次性的获取所有的数据，该方法返回的是一个元组列表。

- 当完成所有操作后，记得断开数据库的连接，释放资源

### 创建游标

返回一个游标Cursor对象

In [2]:
import pymysql
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='python')

cursor = connection.cursor()

print(cursor)

connection.close()

#### 游标对象支持的方法

execute() 执行一个数据库查询和命令

fetchone() 取得结果集的下一行

fetchmany(size) 获取结果集的下几行

fetchall() 返回所有结果集

rowcount 最后一次execute返回数据的行数或影响的行数

close() 关闭游标对象

In [8]:
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')

cursor = connection.cursor()

# 查看数据库中的表
cursor.execute('show tables')
print(cursor.fetchall())

cursor.execute('select version()')
print(cursor.fetchall())

cursor.close()

connection.close()

(('departments',), ('employees',), ('jobs',), ('locations',))
(('8.0.19-0ubuntu0.19.10.3',),)


#### 字典游标

默认游标返回的数据结构是嵌套的元组，然而pymysql提供字典游标，这样我们可以通过列名获取数据

设置方法:

pymysql.connect()中设置cursorclass参数为pymysql.cursors.DictCursor)

In [16]:
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8',
                cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

sql = "select * from employees where salary > 10000"

cursor.execute(sql)

print(cursor.fetchall())

cursor.close()
connection.close()

[{'employee_id': 100, 'first_name': 'Steven', 'last_name': 'K_ing', 'email': 'SKING', 'phone_number': '515.123.4567', 'job_id': 'AD_PRES', 'salary': 24000.0, 'commission_pct': None, 'manager_id': None, 'department_id': 90, 'hiredate': datetime.datetime(1992, 4, 3, 0, 0)}, {'employee_id': 101, 'first_name': 'Neena', 'last_name': 'Kochhar', 'email': 'NKOCHHAR', 'phone_number': '515.123.4568', 'job_id': 'AD_VP', 'salary': 17000.0, 'commission_pct': None, 'manager_id': 100, 'department_id': 90, 'hiredate': datetime.datetime(1992, 4, 3, 0, 0)}, {'employee_id': 102, 'first_name': 'Lex', 'last_name': 'De Haan', 'email': 'LDEHAAN', 'phone_number': '515.123.4569', 'job_id': 'AD_VP', 'salary': 17000.0, 'commission_pct': None, 'manager_id': 100, 'department_id': 90, 'hiredate': datetime.datetime(1992, 4, 3, 0, 0)}, {'employee_id': 108, 'first_name': 'Nancy', 'last_name': 'Greenberg', 'email': 'NGREENBE', 'phone_number': '515.124.4569', 'job_id': 'FI_MGR', 'salary': 12000.0, 'commission_pct': None

## python操作mysql

### 建表

In [11]:
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')

cursor = connection.cursor()

sql = '''
    
    create table python(
        id int auto_increment primary key,
        name char(20) not null,
        sex char(8) not null,
        score int
    )

'''

cursor.execute(sql)

cursor.execute('show tables')
print(cursor.fetchall())

cursor.close()

connection.close()

(('departments',), ('employees',), ('jobs',), ('locations',), ('python',))


### 增

In [20]:

connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')

cursor = connection.cursor()

sql = '''
    insert into python(
    id,name,sex,score
    )
    values(
    null,'sunwei','girl','88'
    )

'''

try:
    cursor.execute(sql)
    
    # 默认情况下，连接不是自动提交
    connection.commit()
    
except:
    connection.rollback()
    
cursor.execute('select * from python')
print(cursor.fetchall())

cursor.close()

connection.close()

((2, 'sunwei', 'girl', 88), (3, 'sunwei', 'girl', 88), (4, 'sunwei', 'girl', 88), (5, 'sunwei', 'girl', 88))


#### 多行插入

使用executemany可以进行多行插入

不管什么类型搭配%s占位符都可以

In [35]:
sql = '''
    insert into python(
    id,name,sex,score
    )
    values(
    %s,%s,%s,%s  
    )

'''

data = (('null','ss','boy',77),
       ('null','xx','boy',66))

try:
    cursor.executemany(sql,data)
    
    # 默认情况下，连接不是自动提交
    connection.commit()
    
except:
    connection.rollback()
    
cursor.execute('select * from python')
print(cursor.fetchall())

cursor.close()

connection.close()

InterfaceError: (0, '')

### 查询

In [28]:
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')

cursor = connection.cursor()

sql = "select * from employees where salary > 10000"

cursor.execute(sql)

print(cursor.rowcount)

cursor.close()
connection.close()

15


### 删除

In [31]:
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')

cursor = connection.cursor()


id1 = input()

# %s作为占位符保证了SQL的安全性
sql = "delete from python where id=%s"

cursor.execute(sql,id1)

connection.commit()

print(cursor.rowcount)

cursor.close()
connection.close()

4
1


### 改

In [32]:
connection = pymysql.connect(host='localhost',
               port=3306,
               user='chenzhi',
               password='cz020391',
               db='myemployees',
               charset='utf8')

cursor = connection.cursor()


id1 = input()

# %s作为占位符保证了SQL的安全性
sql = "update python set sex='boy' where id=%s"

cursor.execute(sql,id1)

connection.commit()

print(cursor.rowcount)

cursor.close()
connection.close()

5
1
